In [2]:
import torch
import deepspeed

checkpoint_dir = "/data2/lzq/mistral-7B-action-level-PPO"
# from deepspeed.utils.zero_to_fp32 import load_state_dict_from_zero_checkpoint
# fp32_model = load_state_dict_from_zero_checkpoint(deepspeed.module, checkpoint_dir)
from deepspeed.utils.zero_to_fp32 import get_fp32_state_dict_from_zero_checkpoint
state_dict = get_fp32_state_dict_from_zero_checkpoint(checkpoint_dir)

Processing zero checkpoint '/data2/lzq/mistral-7B-action-level-PPO/global_step12'
Detected checkpoint of type zero stage 3, world_size: 4
Parsing checkpoint created by deepspeed==0.15.1
Reconstructed Trainable fp32 state dict with 293 params 7242149988 elements


In [3]:
from mistral_with_ppo_head import MistralPPOHeadModel
from transformers import LlamaTokenizerFast, AutoTokenizer, AutoModelForCausalLM, MistralConfig
from ppo_head import PPOHead
model_path = "/share/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/"
config = MistralConfig.from_pretrained(model_path)
agent_model = MistralPPOHeadModel(config=config)
agent_model_embedding_layer = agent_model.model.embed_tokens
embedding_dim = agent_model.model.embed_tokens.embedding_dim
agent_tokenizer = AutoTokenizer.from_pretrained(model_path)
agent_model = agent_model.from_pretrained(model_path, torch_dtype=torch.float16)

if agent_tokenizer.pad_token is None:
    agent_tokenizer.add_special_tokens({'pad_token': '<PAD>'})
    agent_model.resize_token_embeddings(len(agent_tokenizer))
agent_model.set_action_head(PPOHead(input_dim=embedding_dim, output_dim=100))

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.28it/s]


In [4]:
agent_model.load_state_dict(state_dict=state_dict)

<All keys matched successfully>